In [1]:
import configparser
import pandas as pd
from sqlalchemy import create_engine

cfg = configparser.ConfigParser()
cfg.read('config.ini')
db = cfg['database']

uri = (
    f"postgresql+psycopg2://{db['user']}:{db['password']}"
    f"@{db['host']}:{db['port']}/{db['dbname']}"
)
engine = create_engine(uri)


In [2]:
sql = """
SELECT
  weather_records.municipality_id,
  municipality_name,
  station_name,
  station_code,
  postal_code,
  date,
  temperature_observed_min,
  temperature_observed_max,
  temperature_observed_avg,
  temperature_forecast_min,
  temperature_forecast_max,
  temperature_forecast_avg,
  humidity_observed_min,
  humidity_observed_max,
  humidity_observed_avg,
  humidity_forecast_min,
  humidity_forecast_max,
  humidity_forecast_avg
FROM weather_records
join municipalities on weather_records.municipality_id = municipalities.municipality_id
ORDER BY date;
"""
df = pd.read_sql(sql, engine, parse_dates=['date'])


In [3]:
print(df.shape)

(10432, 18)


In [5]:
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interactive_output
from IPython.display import display
import pandas as pd
from typing import Tuple

# Map between display names and DataFrame columns
SERIES_MAP: dict[str, Tuple[str, str]] = {
    "Temperature (Min)": ("temperature_observed_min", "temperature_forecast_min"),
    "Temperature (Max)": ("temperature_observed_max", "temperature_forecast_max"),
    "Temperature (Avg)": ("temperature_observed_avg", "temperature_forecast_avg"),
    "Humidity (Min)":    ("humidity_observed_min",    "humidity_forecast_min"),
    "Humidity (Max)":    ("humidity_observed_max",    "humidity_forecast_max"),
    "Humidity (Avg)":    ("humidity_observed_avg",    "humidity_forecast_avg"),
}

def get_series_columns(label: str) -> Tuple[str, str]:
    """Return observed and forecast column names for a given series label."""
    return SERIES_MAP[label]

def create_figure(
    data: pd.DataFrame,
    municipality: str,
    observed_col: str,
    forecast_col: str,
    series_label: str
) -> go.Figure:
    """Generate a Plotly figure for observed vs forecast data."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=data["date"], y=data[observed_col],
        mode="lines", name="Observed",
        line=dict(color="#1f77b4")
    ))
    fig.add_trace(go.Scatter(
        x=data["date"], y=data[forecast_col],
        mode="lines", name="Forecast",
        line=dict(color="#7f7f7f", dash="dash")
    ))
    fig.update_layout(
        title=f"{series_label} — {municipality}",
        xaxis_title="Date",
        yaxis_title=series_label,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(l=40, r=20, t=60, b=40)
    )
    return fig

# Initialise widgets
municipality_widget = widgets.Dropdown(
    options=sorted(df["municipality_name"].unique()),
    description="Municipality:"
)
series_widget = widgets.Dropdown(
    options=list(SERIES_MAP.keys()),
    description="Series:"
)
start_date_widget = widgets.DatePicker(
    description="Start date",
    value=df["date"].min().date()
)
end_date_widget = widgets.DatePicker(
    description="End date",
    value=df["date"].max().date()
)

def update_plot(
    municipality: str,
    series_label: str,
    start_date,
    end_date
):
    """Callback to update the plot based on widget values."""
    observed_col, forecast_col = get_series_columns(series_label)
    mask = (
        (df["municipality_name"] == municipality) &
        (df["date"] >= pd.to_datetime(start_date)) &
        (df["date"] <= pd.to_datetime(end_date))
    )
    subset = df.loc[mask, ["date", observed_col, forecast_col]].sort_values("date")
    fig = create_figure(subset, municipality, observed_col, forecast_col, series_label)
    fig.show()

# Link widgets to the callback
output = interactive_output(
    update_plot,
    {
        "municipality": municipality_widget,
        "series_label": series_widget,
        "start_date": start_date_widget,
        "end_date": end_date_widget
    }
)

# Arrange and display
controls = widgets.VBox([
    widgets.HBox([municipality_widget, series_widget]),
    widgets.HBox([start_date_widget, end_date_widget])
])
display(widgets.VBox([controls, output]))
